In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 49673580
EPOCHS = 50
size = 64
batchsize = 680
lw= 6
STEPS = TOT / EPOCHS / batchsize / 3
NCATS = 340

In [3]:
K.clear_session()
# base_model = MobileNet(input_shape=(size, size, 1), alpha=1., weights=None, include_top=False)
# x = base_model.output
# # x = GlobalAveragePooling2D()(x)
# x = Flatten()(x)
# x = Dropout(0.5)(x)

# x = Dense(512)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# x = Dense(64)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# predictions = Dense(NCATS, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

# sgd = SGD(0.002, momentum=0.9, nesterov=True)
# sgd = NormalizedOptimizer(sgd, normalization='l2')

model = MobileNet(input_shape=(size, size, 1), alpha=1., weights=None, classes=NCATS)
model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 65, 65, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 32, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 32, 32, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 32, 32, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 32, 32, 32)        128       
__________

In [4]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [5]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 64, 64, 1) (34000, 340)
Validation array memory 0.52 GB


In [6]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, lw=lw,
                                   preprocess_input=preprocess_input)

In [7]:
model_prefix = 'mobilenet_head'
fold = 9
snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,
                                   nb_snapshots=1,
                                   size = size,
                                   init_lr=1e-3, 
                                   fold=fold)

callbks = snapshot.get_callbacks(model_prefix = model_prefix)

In [8]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Stochastic weight averaging selected for last 15 epochs.
Epoch 1/50
487/486 [==============================] - 102s 209ms/step - loss: 3.7632 - categorical_crossentropy: 3.7632 - categorical_accuracy: 0.2104 - top_3_accuracy: 0.3586 - val_loss: 8.5372 - val_categorical_crossentropy: 8.5372 - val_categorical_accuracy: 0.0440 - val_top_3_accuracy: 0.1072

Epoch 00001: val_categorical_accuracy improved from -inf to 0.04397, saving model to ./models/mobilenet_head64_9.model
Epoch 2/50
487/486 [==============================] - 97s 199ms/step - loss: 2.1402 - categorical_crossentropy: 2.1402 - categorical_accuracy: 0.4849 - top_3_accuracy: 0.6953 - val_loss: 3.7132 - val_categorical_crossentropy: 3.7132 - val_categorical_accuracy: 0.3082 - val_top_3_accuracy: 0.4966

Epoch 00002: val_categorical_accuracy improved from 0.04397 to 0.30824, saving model to ./models/mobilenet_head64_9.model
Epoch 3/50
487/486 [==============================] - 98s 201ms/step - loss: 1.7846 - categorical_crossen

487/486 [==============================] - 97s 200ms/step - loss: 1.0506 - categorical_crossentropy: 1.0506 - categorical_accuracy: 0.7318 - top_3_accuracy: 0.8850 - val_loss: 2.9473 - val_categorical_crossentropy: 2.9473 - val_categorical_accuracy: 0.3792 - val_top_3_accuracy: 0.5745

Epoch 00022: val_categorical_accuracy did not improve from 0.70829
Epoch 23/50
487/486 [==============================] - 97s 200ms/step - loss: 1.0471 - categorical_crossentropy: 1.0471 - categorical_accuracy: 0.7319 - top_3_accuracy: 0.8855 - val_loss: 4.1298 - val_categorical_crossentropy: 4.1298 - val_categorical_accuracy: 0.2070 - val_top_3_accuracy: 0.3571

Epoch 00023: val_categorical_accuracy did not improve from 0.70829
Epoch 24/50
487/486 [==============================] - 97s 200ms/step - loss: 1.0332 - categorical_crossentropy: 1.0332 - categorical_accuracy: 0.7359 - top_3_accuracy: 0.8871 - val_loss: 3.9803 - val_categorical_crossentropy: 3.9803 - val_categorical_accuracy: 0.2232 - val_top_3

487/486 [==============================] - 98s 200ms/step - loss: 0.9152 - categorical_crossentropy: 0.9152 - categorical_accuracy: 0.7648 - top_3_accuracy: 0.9036 - val_loss: 0.9088 - val_categorical_crossentropy: 0.9088 - val_categorical_accuracy: 0.7689 - val_top_3_accuracy: 0.9022

Epoch 00043: val_categorical_accuracy improved from 0.76876 to 0.76891, saving model to ./models/mobilenet_head64_9.model
Epoch 44/50
487/486 [==============================] - 98s 200ms/step - loss: 0.9075 - categorical_crossentropy: 0.9075 - categorical_accuracy: 0.7675 - top_3_accuracy: 0.9041 - val_loss: 0.9068 - val_categorical_crossentropy: 0.9068 - val_categorical_accuracy: 0.7693 - val_top_3_accuracy: 0.9033

Epoch 00044: val_categorical_accuracy improved from 0.76891 to 0.76929, saving model to ./models/mobilenet_head64_9.model
Epoch 45/50
487/486 [==============================] - 97s 200ms/step - loss: 0.9056 - categorical_crossentropy: 0.9056 - categorical_accuracy: 0.7678 - top_3_accuracy: 0

In [9]:
prefix_swa = ''
try:
    print('using swa weight model')
    model.load_weights('./models/{}{}_swa_{}.model'.format(model_prefix, size, fold))
    prefix_swa = 'swa'
except:
    model.load_weights('./models/{}{}_{}.model'.format(model_prefix, size, fold))

using swa weight model


In [11]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, preprocess_input=preprocess_input)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

(112199, 3) (112199, 64, 64, 1)
Test array memory 1.71 GB


In [12]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

112199/112199 [==============================] - 11s 100us/step


(112199, 3)

In [13]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

(112199, 3)

In [14]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

,key_id,word
0,9000003627287624,radio stereo snorkel
1,9000010688666847,hockey_puck bottlecap steak
2,9000023642890129,The_Great_Wall_of_China castle camel
3,9000038588854897,mountain tent The_Eiffel_Tower
4,9000052667981386,fireplace campfire crown


In [15]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}{}{}2.csv'.format(model_prefix, size, prefix_swa), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)

save result
upload result
cmd: kaggle competitions submit -c quickdraw-doodle-recognition -f ../result/mobilenet_head64swa2.csv.7z -m "submit"
